In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer
import numpy as np


In [3]:
# Define the dataset
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}
df = pd.DataFrame(ratings_dict)

# Create a user-item matrix
user_item_matrix = df.pivot(index="user", columns="item", values="rating")


In [4]:
# Fill missing values with the mean rating for each user
imputer = SimpleImputer(strategy="mean")
user_item_matrix_filled = imputer.fit_transform(user_item_matrix)

# Compute cosine similarity between items
item_similarity = cosine_similarity(user_item_matrix_filled.T)


In [5]:
# Convert to DataFrame for easier manipulation
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)


In [6]:
# Define a function to recommend items for a given user
def recommend_items(user_id, user_item_matrix, item_similarity_df, n_recommendations=2):
    user_ratings = user_item_matrix.loc[user_id]
    scores = item_similarity_df.dot(user_ratings.fillna(0))  # Weighted sum of item similarities
    scores = scores[user_ratings.isna()]  # Consider only items not yet rated by the user
    recommendations = scores.sort_values(ascending=False).head(n_recommendations)
    return recommendations

# Get recommendations for user 'E'
user_item_matrix = user_item_matrix.fillna(np.nan)  # Recreate the matrix with NaN for better indexing
recommended_items = recommend_items("E", user_item_matrix, item_similarity_df, n_recommendations=2)

print("Recommended movies for user 'E':")
for item_id, score in recommended_items.items():
    print(f"Movie {item_id}: Predicted score {score:.2f}")


Recommended movies for user 'E':
Movie 2: Predicted score 2.92
